Here we bulk process the images to transform it to a ratio of 2:1, where width is 2 and height is 1. Additionally, we would like the horizon line to be at the center of the padded image. 

To calculate the amount of padding, let $W$ and $H$ be the width and height of the original image, $h$ be the depth of the horizon line from the top of the unpadded image, and let $t$ and $b$ be the padding added to the top. Thus we need $t+H+b = \frac{W}{2}$ and $t+h = H-h+b$, perfroming gaussian elimination we get
$$t = \frac{W-4h}{4}, b = \frac{W-4H+4h}{4}$$

As for the interpolated values, we would linearly interpolate up the column from the value of the pixel on the first row of the original image to 255 on the first row of the padded image. For the case of a single channel and a single column, suppose we would like the value of the $i^{th}$ pixel from the top, 
$$\frac{t-i}{t}=\frac{v[i]-im[t]}{255-im[t]}$$

In [ ]:
import wget
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

csv_path = "./data/M2020-Sites - Stereo Mosacis.csv"
data = pd.read_csv(csv_path)

fail_criteria_1 = []
fail_criteria_2 = []
fail_criteria_3 = []
num_succ = 0

for idx in range(data.shape[0]):
  # read the excel
  left_img_url = data["Left"][idx]
  right_img_url = data["Right"][idx]
  name = data["Site name"][idx]

  print(f"{idx}: {name}")

  # download the image
  left_im_path = f"./assets/stereos/unprocessed/{name}_left.jpg"
  right_im_path = f"./assets/stereos/unprocessed/{name}_right.jpg"
  left_im_path = wget.download(left_img_url, out=left_im_path)
  right_im_path = wget.download(right_img_url, out=right_im_path)


  # change color from BGR to RGB
  left_img = np.array(cv2.cvtColor(cv2.imread(left_im_path), cv2.COLOR_BGR2RGB))
  right_img = np.array(cv2.cvtColor(cv2.imread(right_im_path), cv2.COLOR_BGR2RGB))

  # parse the dimensions of the original image
  H, W, C = left_img.shape
  h = data["Horizon"][idx]
  if (W/2) < H:
    fail_criteria_1.append(name)
    continue
  elif W < 4*h:
    fail_criteria_2.append(name)
    continue
  elif W + 4*h < 4*H:
    fail_criteria_3.append(name)
    new_h = H - (W/4)
    print(f"old h:{h}; new h:{new_h}")
    h = new_h
    continue

  # calculate the padding
  t = (W-4*h)/4
  b = (W-4*H+4*h)/4


  # left images
  # for each channel, pad t rows of pixels above and b rows of pixels below
  left_img = np.pad(left_img, ((int(t), int(b)), (0, 0), (0, 0)), mode='constant', constant_values=0)

  # interpolate the top pixels
  for i in range(int(t)):
    left_img[i, :, :] = left_img[int(t), :, :] + (1-i/t)*(255 - left_img[int(t), :, :])

  # downsample the image to 8Kx4K and 16Kx8K
  left_img_8K = cv2.resize(left_img, (2**13, 2**12), interpolation=cv2.INTER_AREA)
  # left_img_16K = cv2.resize(left_img, (2**14, 2**13), interpolation=cv2.INTER_AREA)

  # save the images
  left_img_8K = Image.fromarray(left_img_8K)
  left_img_8K.save(f"./assets/stereos/processed_8K/{name}_left.jpg")
  # left_img_16K = Image.fromarray(left_img_16K)
  # left_img_16K.save(f"./assets/stereos/processed_16K/{name}_left.jpg")

  # right images
  # for each channel, pad t rows of pixels above and b rows of pixels below
  right_img = np.pad(right_img, ((int(t), int(b)), (0, 0), (0, 0)), mode='constant', constant_values=0)

  # interpolate the top pixels
  for i in range(int(t)):
    right_img[i, :, :] = right_img[int(t), :, :] + (1-i/t)*(255 - right_img[int(t), :, :])

  # downsample the image to 8Kx4K and 16Kx8K
  right_img_8K = cv2.resize(right_img, (2**13, 2**12), interpolation=cv2.INTER_AREA)
  # right_img_16K = cv2.resize(right_img, (2**14, 2**13), interpolation=cv2.INTER_AREA)

  # save the images
  right_img_8K = Image.fromarray(right_img_8K)
  right_img_8K.save(f"./assets/stereos/processed_8K/{name}_right.jpg")
  # right_img_16K = Image.fromarray(right_img_16K)
  # right_img_16K.save(f"./assets/stereos/processed_16K/{name}_right.jpg")

  # delete the large donwl images
  os.remove(left_im_path)
  os.remove(right_im_path)

  num_succ += 1

print('-'*100)
print(f"num succ:{num_succ}")
print(f"fail criteria 1:{fail_criteria_1}")
print(f"fail criteria 2:{fail_criteria_2}")
print(f"fail criteria 3:{fail_criteria_3}")

Here we process the hiddn mountain images. The horizon line is 440px from the top and we need to shift the right mosaic 80px to the left.

In [20]:
import numpy as np
import cv2
from PIL import Image

left_img_path = "assets/stereos/unprocessed/N_LRGB_0639XRZS_0310000_CYP_R_AUTOGENJ01_EXPORT.png"
right_img_path = "assets/stereos/unprocessed/N_RRGB_0639XRZS_0310000_CYP_R_AUTOGENJ01_EXPORT.png"

left_img = np.array(cv2.cvtColor(cv2.imread(left_img_path), cv2.COLOR_BGR2RGB))

left_img[0:4, :, :] = left_img[4:8, :, :]
left_img[:,-3:,:] = left_img[:,-6:-3,:]

# parse the dimensions of the original image
H, W, C = left_img.shape
h = 440

if (W/2) < H:
  print("W/2 < H")
elif W < 4*h:
  print("W < 4*h")    
elif W + 4*h < 4*H:
  print("W + 4*h < 4*H")
  h = H - (W/4)

# calculate the padding
t = (W-4*h)/4
b = (W-4*H+4*h)/4

# pad the left image
left_img = np.pad(left_img, ((int(t), int(b)), (0, 0), (0, 0)), mode='constant', constant_values=0)

# interpolate the top pixels
for i in range(int(t)):
  left_img[i, :, :] = left_img[int(t)+5, :, :] + (1-i/t)*(255 - left_img[int(t)+5, :, :])

# downsample the image to 8Kx4K and 16Kx8K
left_img_8K = cv2.resize(left_img, (2**13, 2**12), interpolation=cv2.INTER_AREA)

# save the images
left_img_8K = Image.fromarray(left_img_8K)
left_img_8K.save(f"./assets/stereos/processed_8K/hidden_mountatin_left.jpg")

# right image
right_img = np.array(cv2.cvtColor(cv2.imread(right_img_path), cv2.COLOR_BGR2RGB))
right_img[0:4, :, :] = right_img[4:8, :, :]
right_img[:,-3:,:] = right_img[:,-6:-3,:]

# take the left 80 pixels, cut and paste them to the right
right_img = np.concatenate((right_img[:, 80:, :], right_img[:, :80, :]), axis=1)

# pad the right image
right_img = np.pad(right_img, ((int(t), int(b)), (0, 0), (0, 0)), mode='constant', constant_values=0)

# interpolate the top pixels
for i in range(int(t)):
  right_img[i, :, :] = right_img[int(t)+5, :, :] + (1-i/t)*(255 - right_img[int(t)+5, :, :])

# downsample the image to 8Kx4K and 16Kx8K
right_img_8K = cv2.resize(right_img, (2**13, 2**12), interpolation=cv2.INTER_AREA)

# save the images
right_img_8K = Image.fromarray(right_img_8K)
right_img_8K.save(f"./assets/stereos/processed_8K/hidden_mountatin_right.jpg")


W + 4*h < 4*H


In [5]:
print(f"W:{W}, H:{H}, t:{t}, b:{b}")

W:4320, H:1576, t:640.0, b:-56.0
